In [ ]:
import torch
from dataset import *
import models as m
from tqdm import tqdm
import matplotlib.pyplot as plt
from torchvision.utils import draw_segmentation_masks
import cv2

In [ ]:
validdataset = SatelliteValidDataset(dataset_path='./dataset/')
validloader = torch.utils.data.DataLoader(validdataset,
                                          batch_size=15,
                                          shuffle=False,
                                          num_workers=16)

In [ ]:
def visualize(images, gts, preds):
    batchs = images.size(0)
    
    images = images.cpu().numpy() # -> [batch, channels, height, width]
    gts = gts.cpu().type(torch.bool)  # -> [batch, height, width]
    preds = preds.cpu().type(torch.bool) # -> [batch, height, width]
    
    
    for i in range(batchs):
        norm_img = cv2.normalize(images[i], None, alpha=0, beta=255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
        norm_img = torch.tensor(norm_img.astype(np.uint8), dtype=torch.uint8)
        
        gt = draw_segmentation_masks(norm_img, gts[i], colors='red')
        pred = draw_segmentation_masks(norm_img, preds[i], colors='red')
        
        plt.figure(figsize=(12,4))
        
        plt.subplot(1, 3, 1)
        plt.title('Image')
        plt.imshow(norm_img.numpy().transpose(1,2,0))
        
        plt.subplot(1, 3, 2)
        plt.title('Ground-Truth')
        plt.imshow(gt.numpy().transpose(1,2,0))
        
        plt.subplot(1, 3, 3)
        plt.title('Pred')
        plt.imshow(pred.numpy().transpose(1,2,0))
        
        plt.show()
    

In [ ]:
model = m.UperNet_InternImage_B().to('cuda')
model.load_state_dict(torch.load('./checkpoint3/UperNet_InternImage_B_10'))

In [ ]:
count = 0
for img, mask in validloader:

    imgf = img.float().to('cuda')
    outputs = model(imgf)
    pred = (torch.sigmoid(outputs) > 0.35).squeeze(1)

    break

In [ ]:
visualize(img, mask, pred)